In [0]:
# 1. Imports e Configuração

# COMMAND ----------

import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from datetime import datetime

spark = SparkSession.builder \
    .appName("SilverCNPJ") \
    .getOrCreate()


In [0]:
# MAGIC ## 2. Configuração de Storage (ABFSS - ADLS Gen2)

# COMMAND ----------

# ✅ Recuperar Storage Account
TGT_STORAGE_ACCOUNT = dbutils.secrets.get(scope="acelera-grupo-5-kv", key="tgt-storage-account")

# ✅ Containers
SILVER_CONTAINER = "silver"

# ✅ USAR ABFSS:// (Azure Data Lake Storage Gen2) - MESMO PROTOCOLO DO BRONZE
SILVER_BASE_PATH = f"abfss://{SILVER_CONTAINER}@{TGT_STORAGE_ACCOUNT}.dfs.core.windows.net/cnpj"
CONTROL_TABLE_PATH = f"abfss://{SILVER_CONTAINER}@{TGT_STORAGE_ACCOUNT}.dfs.core.windows.net/metadata/silver_control_cnpj"

print(f"✅ Storage Account: {TGT_STORAGE_ACCOUNT}")
print(f"✅ Silver Container: {SILVER_CONTAINER}")
print(f"✅ Protocolo: abfss:// (ADLS Gen2)")
print(f"📂 Silver Base Path: {SILVER_BASE_PATH}")
print(f"📂 Control Table Path: {CONTROL_TABLE_PATH}")
print(f"🗄️ Bronze: Leitura via Hive Metastore (bronze)")

# COMMAND ----------

# 🧪 VALIDAR ACESSO ao container Silver
print("\n🧪 Validando acesso ao container silver...")

try:
    test_path = f"abfss://{SILVER_CONTAINER}@{TGT_STORAGE_ACCOUNT}.dfs.core.windows.net/"
    files = dbutils.fs.ls(test_path)
    print(f"✅ Container 'silver' acessível - {len(files)} itens encontrados")
    
except Exception as e:
    print(f"❌ ERRO: Cluster não tem acesso ao container 'silver'")
    print(f"\n⚠️ Verifique:")
    print(f"   1. Container 'silver' existe no Storage Account")
    print(f"   2. Service Principal/Managed Identity tem permissões")
    print(f"   3. Role necessária: 'Storage Blob Data Contributor'")
    print(f"\nErro: {str(e)[:300]}")
    raise

In [0]:
# MAGIC ## 3. Otimizações Spark

# COMMAND ----------

spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")

In [0]:
# MAGIC ## 4. Criação de Databases

# COMMAND ----------

spark.sql("CREATE DATABASE IF NOT EXISTS silver")
spark.sql("CREATE DATABASE IF NOT EXISTS metadata")

print("✅ Databases criados no Hive Metastore")

In [0]:
# MAGIC ## 5. Criação da Tabela de Controle

# COMMAND ----------

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS metadata.silver_control_cnpj (
        table_name STRING COMMENT 'Nome da tabela Silver',
        last_ingestion_timestamp TIMESTAMP COMMENT 'Último timestamp processado',
        last_update TIMESTAMP COMMENT 'Timestamp da última atualização',
        records_processed BIGINT COMMENT 'Total de registros processados',
        execution_id STRING COMMENT 'ID da execução'
    )
    USING DELTA
    LOCATION '{CONTROL_TABLE_PATH}'
    COMMENT 'Controle incremental - Silver CNPJ'
""")

print("✅ Tabela de controle criada")

# Verificar location
location = spark.sql("DESCRIBE DETAIL metadata.silver_control_cnpj").select("location").first()[0]
print(f"📂 Location: {location}")

In [0]:
# MAGIC ## 6. Configuração de Transformações

# COMMAND ----------

table_config = [
    # ========================
    # CNAE
    # ========================
    {
        "bronze_table": "cnaes",
        "silver_table": "cnaes",
        "key_columns": ["codigo_cnae"],
        "column_mapping": {
            "codigo_cnae": {"rename": "codigo_cnae", "cast": "string", "trim": False},
            "descricao_cnae": {"rename": "descricao_cnae", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Classificação Nacional de Atividades Econômicas (CNAE)"
    },

    # ========================
    # EMPRESAS
    # ========================
    {
        "bronze_table": "empresas",
        "silver_table": "empresas",
        "key_columns": ["cnpj_basico"],
        "column_mapping": {
            "cnpj_basico": {"rename": "cnpj_basico", "cast": "string", "trim": False},
            "razao_social": {"rename": "razao_social", "cast": "string", "trim": True},
            "natureza_juridica": {"rename": "natureza_juridica", "cast": "string", "trim": False},
            "qualificacao_responsavel": {"rename": "qualificacao_responsavel", "cast": "string", "trim": False},
            "capital_social": {"rename": "capital_social", "cast": "decimal(18,2)", "trim": False},
            "porte_empresa": {"rename": "porte_empresa", "cast": "string", "trim": False},
            "ente_federativo_responsavel": {"rename": "ente_federativo_responsavel", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Dados cadastrais das empresas (nível CNPJ básico)"
    },

    # ========================
    # ESTABELECIMENTOS
    # ========================
    {
        "bronze_table": "estabelecimentos",
        "silver_table": "estabelecimentos",
        "key_columns": ["cnpj_basico", "cnpj_ordem", "cnpj_dv"],
        "column_mapping": {
            "cnpj_basico": {"rename": "cnpj_basico", "cast": "string", "trim": False},
            "cnpj_ordem": {"rename": "cnpj_ordem", "cast": "string", "trim": False},
            "cnpj_dv": {"rename": "cnpj_dv", "cast": "string", "trim": False},
            "identificador_matriz_filial": {"rename": "identificador_matriz_filial", "cast": "string", "trim": False},
            "nome_fantasia": {"rename": "nome_fantasia", "cast": "string", "trim": True},
            "situacao_cadastral": {"rename": "situacao_cadastral", "cast": "string", "trim": False},
            "data_situacao_cadastral": {"rename": "data_situacao_cadastral", "cast": "date", "trim": False},
            "motivo_situacao_cadastral": {"rename": "motivo_situacao_cadastral", "cast": "int", "trim": False},
            "nome_cidade_exterior": {"rename": "nome_cidade_exterior", "cast": "string", "trim": True},
            "pais": {"rename": "pais", "cast": "int", "trim": False},
            "data_inicio_atividade": {"rename": "data_inicio_atividade", "cast": "date", "trim": False},
            "cnae_fiscal_principal": {"rename": "cnae_fiscal_principal", "cast": "string", "trim": False},
            "cnae_fiscal_secundaria": {"rename": "cnae_fiscal_secundaria", "cast": "string", "trim": False},
            "tipo_logradouro": {"rename": "tipo_logradouro", "cast": "string", "trim": True},
            "logradouro": {"rename": "logradouro", "cast": "string", "trim": True},
            "numero": {"rename": "numero", "cast": "string", "trim": True},
            "complemento": {"rename": "complemento", "cast": "string", "trim": True},
            "bairro": {"rename": "bairro", "cast": "string", "trim": True},
            "cep": {"rename": "cep", "cast": "string", "trim": False},
            "uf": {"rename": "uf", "cast": "string", "trim": False},
            "municipio": {"rename": "municipio", "cast": "string", "trim": False},
            "ddd_1": {"rename": "ddd1", "cast": "string", "trim": False},
            "telefone_1": {"rename": "telefone1", "cast": "string", "trim": False},
            "ddd_2": {"rename": "ddd2", "cast": "string", "trim": False},
            "telefone_2": {"rename": "telefone2", "cast": "string", "trim": False},
            "ddd_fax": {"rename": "ddd_fax", "cast": "string", "trim": False},
            "fax": {"rename": "fax", "cast": "string", "trim": False},
            "email": {"rename": "email", "cast": "string", "trim": True},
            "situacao_especial": {"rename": "situacao_especial", "cast": "string", "trim": True},
            "data_situacao_especial": {"rename": "data_situacao_especial", "cast": "date", "trim": False}
        },
        "partition_by": ["uf"],
        "description": "Estabelecimentos vinculados ao CNPJ (matriz e filiais)"
    },

    # ========================
    # MOTIVOS
    # ========================
    {
        "bronze_table": "motivos",
        "silver_table": "motivos",
        "key_columns": ["codigo_motivo"],
        "column_mapping": {
            "codigo_motivo": {"rename": "codigo_motivo", "cast": "string", "trim": False},
            "descricao_motivo": {"rename": "descricao_motivo", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Motivos de situação cadastral"
    },

    # ========================
    # MUNICÍPIOS
    # ========================
    {
        "bronze_table": "municipios",
        "silver_table": "municipios",
        "key_columns": ["codigo_municipio"],
        "column_mapping": {
            "codigo_municipio": {"rename": "codigo_municipio", "cast": "string", "trim": False},
            "descricao_municipio": {"rename": "descricao_municipio", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Cadastro de municípios do Brasil"
    },

    # ========================
    # NATUREZAS JURÍDICAS
    # ========================
    {
        "bronze_table": "naturezas",
        "silver_table": "naturezas",
        "key_columns": ["codigo_natureza_juridica"],
        "column_mapping": {
            "codigo_natureza_juridica": {"rename": "codigo_natureza_juridica", "cast": "string", "trim": False},
            "descricao_natureza_juridica": {"rename": "descricao_natureza_juridica", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Naturezas jurídicas das empresas"
    },

    # ========================
    # PAÍSES
    # ========================
    {
        "bronze_table": "paises",
        "silver_table": "paises",
        "key_columns": ["codigo_pais"],
        "column_mapping": {
            "codigo_pais": {"rename": "codigo_pais", "cast":"string","trim": False},
            "descricao_pais": {"rename": "descricao_pais", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Cadastro de países"
    },

    # ========================
    # QUALIFICAÇÕES
    # ========================
    {
        "bronze_table": "qualificacoes",
        "silver_table": "qualificacoes",
        "key_columns": ["codigo_qualificacao"],
        "column_mapping": {
            "codigo_qualificacao": {"rename": "codigo_qualificacao", "cast": "string", "trim": False},
            "descricao_qualificacao": {"rename": "descricao_qualificacao", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Qualificações de sócios e responsáveis legais"
    },

    # ========================
    # SIMPLES / MEI
    # ========================
    {
        "bronze_table": "simples",
        "silver_table": "simples",
        "key_columns": ["cnpj_basico"],
        "column_mapping": {
            "cnpj_basico": {"rename": "cnpj_basico", "cast": "string", "trim": False},
            "opcao_simples": {"rename": "opcao_simples", "cast": "string", "trim": False},
            "data_opcao_simples": {"rename": "data_opcao_simples", "cast": "date", "trim": False},
            "data_exclusao_simples": {"rename": "data_exclusao_simples", "cast": "date", "trim": False},
            "opcao_mei": {"rename": "opcao_mei", "cast": "string", "trim": False},
            "data_opcao_mei": {"rename": "data_opcao_mei", "cast": "date", "trim": False},
            "data_exclusao_mei": {"rename": "data_exclusao_mei", "cast": "date", "trim": False}
        },
        "partition_by": None,
        "description": "Opção pelo Simples Nacional e MEI"
    },

    # ========================
    # SÓCIOS
    # ========================
    {
        "bronze_table": "socios",
        "silver_table": "socios",
        "key_columns": ["cnpj_basico", "cpf_cnpj_socio"],
        "column_mapping": {
            "cnpj_basico": {"rename": "cnpj_basico", "cast": "string", "trim": False},
            "identificador_socio": {"rename": "identificador_socio", "cast": "string", "trim": False},
            "nome_socio_razao_social": {"rename": "nome_socio_razao_social", "cast": "string", "trim": True},
            "cpf_cnpj_socio": {"rename": "cpf_cnpj_socio", "cast": "string", "trim": False},
            "qualificacao_socio": {"rename": "qualificacao_socio", "cast": "string", "trim": False},
            "data_entrada_sociedade": {"rename": "data_entrada_sociedade", "cast": "date", "trim": False},
            "pais": {"rename": "pais", "cast": "string", "trim": False},
            "cpf_representante_legal": {"rename": "cpf_representante_legal", "cast": "string", "trim": True},
            "nome_representante_legal": {"rename": "nome_representante_legal", "cast": "string", "trim": True},
            "qualificacao_representante_legal": {"rename": "qualificacao_representante_legal", "cast": "string", "trim": False},
            "faixa_etaria": {"rename": "faixa_etaria", "cast": "int", "trim": False}
        },
        "partition_by": None,
        "description": "Quadro societário das empresas"
    }
]

print(f"✅ {len(table_config)} tabelas configuradas")

In [0]:
# MAGIC ## 7. Funções de Transformação

# COMMAND ----------

def get_last_processed_timestamp(silver_table_name: str) -> datetime:
    """
    Recupera último timestamp processado da tabela de controle
    
    Args:
        silver_table_name: Nome da tabela Silver
        
    Returns:
        Último timestamp processado ou None se primeira execução
    """
    try:
        row = (
            spark.table("metadata.silver_control_cnpj")
            .filter(F.col("table_name") == silver_table_name)
            .select("last_ingestion_timestamp")
            .orderBy(F.col("last_update").desc())
            .limit(1)
            .collect()
        )
        
        if row:
            timestamp = row[0]["last_ingestion_timestamp"]
            print(f"📅 Último processamento: {timestamp}")
            return timestamp
        else:
            print(f"🆕 Primeira execução")
            return None
            
    except Exception as e:
        print(f"⚠️ Controle vazio: {e}")
        return None


def get_bronze_incremental_latest(bronze_table: str, pk_columns: list, last_processed_at: datetime):
    """
    Lê dados incrementais do Bronze via HIVE METASTORE
    
    Args:
        bronze_table: Nome da tabela Bronze (sem database)
        pk_columns: Lista de colunas da chave primária
        last_processed_at: Timestamp do último processamento (watermark)
        
    Returns:
        DataFrame com dados novos (DEDUPLICAÇÃO SERÁ FEITA DEPOIS DA TRANSFORMAÇÃO)
    """
    
    # ✅ Leitura via Hive Metastore
    full_table_name = f"bronze.{bronze_table}"
    
    try:
        df_bronze = spark.table(full_table_name)
    except Exception as e:
        print(f"❌ Tabela Bronze não existe: {full_table_name}")
        raise
    
    # Filtro incremental usando _ingestion_timestamp (nome correto do Bronze)
    if last_processed_at:
        df_filtered = df_bronze.filter(F.col("_ingestion_timestamp") > F.lit(last_processed_at))
        records_filtered = df_filtered.count()
        print(f"📊 Registros incrementais: {records_filtered:,}")
    else:
        df_filtered = df_bronze
        records_total = df_filtered.count()
        print(f"📊 Carga FULL: {records_total:,}")
    
    # ✅ RETORNA SEM DEDUPLICAÇÃO
    # A deduplicação será feita DEPOIS da transformação de colunas
    # para evitar conflito entre nomes Bronze e Silver
    
    return df_filtered


def apply_column_mapping(df, column_mapping: dict):
    """
    Aplica transformações de colunas (cast, rename, trim)
    
    Args:
        df: DataFrame Bronze
        column_mapping: Dicionário com regras de transformação
        
    Returns:
        DataFrame transformado
    """
    select_exprs = []
    
    for source_col, rules in column_mapping.items():
        if source_col not in df.columns:
            print(f"⚠️ Coluna {source_col} não encontrada - ignorando")
            continue
        
        # Suporte para mapeamento simples (string)
        if isinstance(rules, str):
            select_exprs.append(F.col(source_col).alias(rules))
            continue
        
        # Mapeamento completo (dict)
        target_col = rules.get("rename", source_col)
        cast_type = rules.get("cast")
        trim_flag = rules.get("trim", False)
        
        col_expr = F.col(source_col)
        
        # Aplicar trim se string
        if trim_flag:
            col_expr = F.trim(col_expr)
        
        # ✅ TRATAMENTO ESPECIAL PARA DATAS (formato AAAAMMDD)
        if cast_type == "date":
            # Converter string AAAAMMDD para DATE
            # Exemplo: "20210315" -> 2021-03-15
            col_expr = F.to_date(col_expr, "yyyyMMdd")
        elif cast_type:
            # Aplicar cast normal para outros tipos
            col_expr = col_expr.cast(cast_type)
        
        select_exprs.append(col_expr.alias(target_col))
    
    # Preservar metadados técnicos (NOMES CORRETOS DO BRONZE)
    technical_cols = ["ingestion_date", "_ingestion_timestamp", "_source_path"]
    for col_name in technical_cols:
        if col_name in df.columns:
            select_exprs.append(F.col(col_name))
    
    # ✅ ADICIONAR TIMESTAMP DE INSERÇÃO NA SILVER
    select_exprs.append(F.current_timestamp().alias("_silver_timestamp"))
    
    return df.select(*select_exprs)


def deduplicate_by_pk(df, pk_columns: list):
    """
    Deduplica DataFrame por chave primária (mantém registro mais recente)
    
    Args:
        df: DataFrame já transformado (com nomes Silver)
        pk_columns: Lista de colunas da chave primária (nomes Silver)
        
    Returns:
        DataFrame deduplicado
    """
    
    # Window por PK, ordenado por timestamp descendente
    window_pk = Window.partitionBy(*pk_columns).orderBy(F.col("_ingestion_timestamp").desc())
    
    df_dedup = (
        df
        .withColumn("_rn", F.row_number().over(window_pk))
        .filter(F.col("_rn") == 1)
        .drop("_rn")
    )
    
    records_dedup = df_dedup.count()
    print(f"✅ Registros únicos após deduplicação: {records_dedup:,}")
    
    return df_dedup


def merge_into_silver(df_source, silver_table_name: str, pk_columns: list, partition_by: list):
    """
    Executa MERGE na Silver e registra no Hive Metastore
    
    Args:
        df_source: DataFrame transformado
        silver_table_name: Nome da tabela Silver (ex: 'silver_empresas')
        pk_columns: Colunas da chave primária
        partition_by: Colunas para particionamento (ou None)
    """
    
    # Path físico no ADLS Gen2
    silver_path = f"{SILVER_BASE_PATH}/{silver_table_name}"
    full_table_name = f"silver.{silver_table_name}"
    
    # Verificar se tabela existe
    if not DeltaTable.isDeltaTable(spark, silver_path):
        print(f"🆕 Criando tabela: {full_table_name}")
        
        # Escrever arquivos Delta
        writer = df_source.write.format("delta").mode("overwrite")
        
        if partition_by:
            writer = writer.partitionBy(*partition_by)
            print(f"📂 Particionamento: {partition_by}")
        
        writer.save(silver_path)
        
        # ✅ Registrar no Hive Metastore
        spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {full_table_name}
            USING DELTA
            LOCATION '{silver_path}'
        """)
        
        print(f"✅ Tabela criada e registrada no Hive")
        return
    
    # MERGE incremental (SCD Type 1)
    print(f"🔄 Executando MERGE: {full_table_name}")
    
    delta_silver = DeltaTable.forPath(spark, silver_path)
    merge_condition = " AND ".join([f"silver.{pk} = source.{pk}" for pk in pk_columns])
    
    # Colunas para update (todas exceto PKs)
    update_columns = {col: f"source.{col}" for col in df_source.columns if col not in pk_columns}
    
    # Colunas para insert (todas)
    insert_columns = {col: f"source.{col}" for col in df_source.columns}
    
    (
        delta_silver.alias("silver")
        .merge(df_source.alias("source"), merge_condition)
        .whenMatchedUpdate(set=update_columns)
        .whenNotMatchedInsert(values=insert_columns)
        .execute()
    )
    
    print(f"✅ MERGE concluído")


def update_control_table(silver_table_name: str, max_timestamp: datetime, records_count: int, execution_id: str):
    """
    Atualiza tabela de controle com watermark
    
    Args:
        silver_table_name: Nome da tabela Silver
        max_timestamp: Timestamp máximo processado
        records_count: Quantidade de registros processados
        execution_id: ID da execução
    """
    
    # ✅ CALCULAR timestamp atual ANTES de criar tupla
    current_ts = datetime.now()
    
    control_data = [(silver_table_name, max_timestamp, current_ts, records_count, execution_id)]
    df_new = spark.createDataFrame(
        control_data,
        ["table_name", "last_ingestion_timestamp", "last_update", "records_processed", "execution_id"]
    )
    
    if DeltaTable.isDeltaTable(spark, CONTROL_TABLE_PATH):
        delta_control = DeltaTable.forPath(spark, CONTROL_TABLE_PATH)
        
        (
            delta_control.alias("t")
            .merge(df_new.alias("s"), "t.table_name = s.table_name")
            .whenMatchedUpdate(set={
                "last_ingestion_timestamp": "s.last_ingestion_timestamp",
                "last_update": "s.last_update",
                "records_processed": "t.records_processed + s.records_processed",
                "execution_id": "s.execution_id"
            })
            .whenNotMatchedInsert(values={
                "table_name": "s.table_name",
                "last_ingestion_timestamp": "s.last_ingestion_timestamp",
                "last_update": "s.last_update",
                "records_processed": "s.records_processed",
                "execution_id": "s.execution_id"
            })
            .execute()
        )
        
        print(f"📝 Controle atualizado")
    else:
        df_new.write.format("delta").mode("overwrite").save(CONTROL_TABLE_PATH)
        print(f"🆕 Controle inicializado")

In [0]:
# MAGIC ## 8. Processamento Incremental

# COMMAND ----------

execution_id = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"🚀 Execution ID: {execution_id}")
print(f"{'='*80}\n")

# Estatísticas da execução
total_tables_processed = 0
total_records_processed = 0
tables_created = []
tables_updated = []
tables_failed = []

for config in table_config:
    
    bronze_table = config["bronze_table"]
    silver_table = config["silver_table"]
    pk_cols = config["key_columns"]
    column_mapping = config["column_mapping"]
    partition_by = config["partition_by"]
    description = config["description"]
    
    print(f"\n{'='*80}")
    print(f"📋 Bronze: bronze.{bronze_table}")
    print(f"📋 Silver: silver.{silver_table}")
    print(f"📝 Descrição: {description}")
    print(f"{'='*80}")
    
    try:
        # 1. Verificar se Bronze existe
        try:
            spark.table(f"bronze.{bronze_table}").limit(1).count()
        except:
            print(f"⚠️ Tabela Bronze não existe: bronze.{bronze_table}")
            tables_failed.append(silver_table)
            continue
        
        # 2. Recuperar último timestamp processado
        last_processed_at = get_last_processed_timestamp(silver_table)
        
        # 3. Ler dados incrementais do Bronze (SEM deduplicação ainda)
        df_latest = get_bronze_incremental_latest(bronze_table, pk_cols, last_processed_at)
        
        # 4. Verificar se há dados novos
        if df_latest.count() == 0:
            print(f"ℹ️ Sem dados novos para processar")
            continue
        
        # 5. Aplicar transformações (rename, cast, trim)
        df_transformed = apply_column_mapping(df_latest, column_mapping)
        
        # 6. ✅ DEDUPLICAR APÓS TRANSFORMAÇÃO (com nomes Silver corretos)
        df_transformed = deduplicate_by_pk(df_transformed, pk_cols)
        
        # 7. Verificar se é tabela nova
        is_new_table = not spark.catalog.tableExists(f"silver.{silver_table}")
        
        # 8. MERGE na Silver
        merge_into_silver(df_transformed, silver_table, pk_cols, partition_by)
        
        # 9. Atualizar controle
        max_timestamp = df_transformed.agg(F.max("_ingestion_timestamp")).collect()[0][0]
        records_count = df_transformed.count()
        
        update_control_table(silver_table, max_timestamp, records_count, execution_id)
        
        # 10. Registrar estatísticas
        total_tables_processed += 1
        total_records_processed += records_count
        
        if is_new_table:
            tables_created.append(silver_table)
        else:
            tables_updated.append(silver_table)
        
        print(f"✅ {silver_table} processado com sucesso!")
        
    except Exception as e:
        print(f"❌ ERRO ao processar {silver_table}")
        print(f"   Erro: {str(e)}")
        tables_failed.append(silver_table)
        
        import traceback
        traceback.print_exc()

print(f"\n{'='*80}")
print(f"🎉 PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")

In [0]:
# MAGIC ## 9. Relatório de Execução

# COMMAND ----------

print(f"""
╔══════════════════════════════════════════════════════════════════╗
║                  RELATÓRIO - SILVER CNPJ                         ║
╚══════════════════════════════════════════════════════════════════╝

📊 ESTATÍSTICAS GERAIS:
   • Execution ID: {execution_id}
   • Tabelas processadas: {total_tables_processed}
   • Registros processados: {total_records_processed:,}
   • Protocolo: abfss:// (ADLS Gen2)

✅ TABELAS CRIADAS ({len(tables_created)}):
{chr(10).join(f'   • {t}' for t in tables_created) if tables_created else '   (nenhuma)'}

🔄 TABELAS ATUALIZADAS ({len(tables_updated)}):
{chr(10).join(f'   • {t}' for t in tables_updated) if tables_updated else '   (nenhuma)'}

{f'❌ TABELAS COM FALHA ({len(tables_failed)}):' if tables_failed else ''}
{chr(10).join(f'   • {t}' for t in tables_failed) if tables_failed else ''}

🗄️ ARQUITETURA:
   • Bronze (leitura): Hive Metastore → bronze_cnpj.*
   • Silver (escrita): ADLS Gen2 + Hive → silver_cnpj.*
   • Controle: metadata.silver_control_cnpj

📂 LOCALIZAÇÃO FÍSICA:
   • Silver Tables: {SILVER_BASE_PATH}
   • Control Table: {CONTROL_TABLE_PATH}

🔗 CONEXÃO POWER BI:
   • Catalog: hive_metastore
   • Schema: silver
   • Autenticação: Configurada no cluster (Service Principal/Managed Identity)

⏱️ PRÓXIMA EXECUÇÃO:
   • Apenas dados com _ingestion_timestamp > último processamento
   • Processamento incremental automático
""")

In [0]:
%sql
DROP TABLE IF EXISTS silver.corr_ncm_cnae;

CREATE TABLE silver.corr_ncm_cnae
USING DELTA
LOCATION 'abfss://silver@aceleragrupo5sa.dfs.core.windows.net/corr_ncm_cnae'
AS SELECT * FROM bronze.corr_ncm_cnae